## Implementation of Metropolis Hastings to approximate probability queries from a Bayesian Network.

In [2]:
import numpy as np
from calculations_helper import break_up_polytree, handle_dag_metropolis_hastings, join_distributions

def estimate_metropolis_hastings(p: float, iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> tuple[list[int], float, np.array]:
    """Generate an estimate for the probability distribution for a given set of query variables and evidence values

    Args:
        p (float): determines probability of generating the next state via either Gibbs Sampling or Likelihood Weighting
        iterations (int): number of samples to take before we go with the estimate
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables with their respective values

    Returns:
        tuple[list[int], float, np.array]: list of variables, attached probability for using Gibbs for state transition, and resulting probability distribution
    """
    dag_map, query_collections, evidence_collections = break_up_polytree(network, queries, evidence)
    
    if len(dag_map) == 1:
        # only one directed acyclic graph
        return queries, handle_dag_metropolis_hastings(iterations, p, network, queries, evidence)
    else:
        # each directed acyclic graph will output a probability distribution - we must join them all and keep track of the variables present
        reordered_queries = []
        prob_distributions = []
        for i, dag in dag_map.items():
            these_queries = query_collections[i]
            for v in these_queries:
                reordered_queries.append(v)
            this_evidence = {v:evidence[v] for v in evidence_collections[i]}
            prob_distributions.append(handle_dag_metropolis_hastings(iterations, p, dag, these_queries, this_evidence))
        return reordered_queries, p, join_distributions(prob_distributions)

In [ ]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('small_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_metropolis_hastings(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

In [6]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('big_polytree.json') as f:
        bayesian_network = json.load(f)
        start_time = time.perf_counter()
        results.append(estimate_metropolis_hastings(10000, bayesian_network, queries, evidence))
        end_time = time.perf_counter()
        times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\nProbabilities={res[1]}\nRuntime={t}\n\n")

TypeError: estimate_metropolis_hastings() missing 1 required positional argument: 'evidence'

In [5]:
import json
import time

query_list = [[1, 3], [4, 5, 7], [2,6,10,12]]
evidence_list = [{2:False, 5:True}, {2:True, 3:False, 6:False}, {3:False, 4:False, 5:True, 15:True}]

times = []
results = []

for queries,evidence in zip(query_list,evidence_list):
    with open('giant_polytree.json') as f:
        bayesian_network = json.load(f)
        for p in [0.75, 0.85, 0.95]:
            start_time = time.perf_counter()
            results.append(estimate_metropolis_hastings(p, 10000, bayesian_network, queries, evidence))
            end_time = time.perf_counter()
            times.append(end_time-start_time)
for res, t in zip(results, times):
    print(f"Variables={res[0]}\np-value={res[1]}\nProbabilities={res[2]}\nRuntime={t}\n\n")

Variables=[1, 3]
p-value=0.75
Probabilities=[0.24639044 0.2999422  0.23218211 0.22148525]
Runtime=9.197118417010643


Variables=[1, 3]
p-value=0.85
Probabilities=[0.2466703  0.28246182 0.23661104 0.23425684]
Runtime=9.421233959001256


Variables=[1, 3]
p-value=0.95
Probabilities=[0.24460401 0.26347447 0.24530656 0.24661496]
Runtime=9.745251499989536


Variables=[4, 5, 7]
p-value=0.75
Probabilities=[0.12532927 0.12405012 0.12357531 0.12063595 0.12567895 0.1211324
 0.12925349 0.13034451]
Runtime=9.310136083004181


Variables=[4, 5, 7]
p-value=0.85
Probabilities=[0.1244595  0.12570517 0.12707016 0.13019385 0.11906085 0.12477376
 0.12032198 0.12841473]
Runtime=9.979918416996952


Variables=[4, 5, 7]
p-value=0.95
Probabilities=[0.13099254 0.12148546 0.12348017 0.12987099 0.1262263  0.12400699
 0.12034985 0.12358771]
Runtime=10.544327499999781


Variables=[2, 6, 10, 12]
p-value=0.75
Probabilities=[0.06376424 0.06242175 0.06270001 0.06176003 0.06565444 0.06295465
 0.06604153 0.05898606 0.0592